### **Sharon Laurance Muthipeedika**
### 312486

### **LeNet MNIST Tensorboard**

In [1]:
#Load the Libraries
import numpy as np
from datetime import datetime 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import gc
# check device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
## Print Device being used
DEVICE

'cuda'

In [ ]:
# parameters
RANDOM_SEED = 42
BATCH_SIZE = 32
N_EPOCHS = 15
IMG_SIZE = 32
N_CLASSES = 10

In [ ]:
# define transforms
transforms = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor()])

# download and create datasets
train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transforms,
                               download=True)

test_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transforms)

# define the data loaders
train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

test_loader = DataLoader(dataset=test_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



In [ ]:
# Class LeNet which contains layers and forward pass- Convolution Block, Linear Block and Forward Pass
class LeNet5(nn.Module):

    def __init__(self, n_classes):
        super(LeNet5, self).__init__()       
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )
        self.classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )
    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        logits = self.classifier(x)
        probs = F.softmax(logits, dim=1)
        return logits, probs


In [ ]:
def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''
    model.train()
    running_loss = 0
    for X, y_true in train_loader:
        optimizer.zero_grad()
        X = X.to(device)
        y_true = y_true.to(device)
        # Forward pass
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)
        # Backward pass
        loss.backward()
        optimizer.step()   
    epoch_loss = running_loss / len(train_loader.dataset)
    return model, optimizer, epoch_loss


In [ ]:
def validate(test_loader, model, criterion, device):
    '''
    Function for the testing step of the training loop
    '''
   
    model.eval()
    running_loss = 0
    
    for X, y_true in test_loader:
    
        X = X.to(device)
        y_true = y_true.to(device)

        # Forward pass and record loss
        y_hat, _ = model(X) 
        loss = criterion(y_hat, y_true) 
        running_loss += loss.item() * X.size(0)

    epoch_loss = running_loss / len(test_loader.dataset)
        
    return model, epoch_loss

In [ ]:
def training_loop(model, criterion, optimizer, train_loader, test_loader, epochs, device,lr, print_every=1):
    '''
    Function defining the entire training loop
    '''
    # set objects for storing metrics
    best_loss = 1e10
    train_losses = []
    test_losses = []
    lr=str(lr)
    # Train model
    for epoch in range(0, epochs):
        # training
        model, optimizer, train_loss = train(train_loader, model, criterion, optimizer, device)
        writer.add_scalar("Epoch/Training_Loss for Learning Rate "+lr, train_loss, epoch)      
        train_losses.append(train_loss)
        # Testing
        with torch.no_grad():
            model, test_loss = validate(test_loader, model, criterion, device)
            test_losses.append(test_loss)
            writer.add_scalar("Epoch/Testing_Loss for Learning Rate "+lr, test_loss, epoch)
        if epoch % print_every == (print_every - 1):           
            train_acc = get_accuracy(model, train_loader, device=device)
            valid_acc = get_accuracy(model, test_loader, device=device)
            writer.add_scalar("Epoch/Accuracy_Training for Learning Rate "+lr, train_acc, epoch)
            writer.add_scalar("Epoch/Accuracy_Testing for Learning Rate "+lr, valid_acc, epoch)              
            print(f'{datetime.now().time().replace(microsecond=0)} --- '
                  f'Epoch: {epoch}\t'
                  f'Train loss: {train_loss:.4f}\t'
                  f'Test loss: {test_loss:.4f}\t'
                  f'Train accuracy: {100 * train_acc:.2f}\t'
                  f'Test accuracy: {100 * valid_acc:.2f}')
    writer.flush()
    return model, optimizer, (train_losses, test_losses)

In [ ]:
def get_accuracy(model, data_loader, device):
    '''
    Function for computing the accuracy of the predictions over the entire data_loader
    '''
    
    correct_pred = 0 
    n = 0
    
    with torch.no_grad():
        model.eval()
        for X, y_true in data_loader:

            X = X.to(device)
            y_true = y_true.to(device)

            _, y_prob = model(X)
            _, predicted_labels = torch.max(y_prob, 1)

            n += y_true.size(0)
            correct_pred += (predicted_labels == y_true).sum()

    return correct_pred.float() / n

In [ ]:
# Adam Optimizer is used and learning rate will be varied as [0.1, 0.01, 0.001]
torch.manual_seed(RANDOM_SEED)
model = LeNet5(N_CLASSES).to(DEVICE)

LEARNING_RATE_arr = [0.1, 0.01, 0.001]
optimizer_arr=[]
for i in LEARNING_RATE_arr:
  optimizer = torch.optim.Adam(model.parameters(), lr=i)
  optimizer_arr.append(optimizer)
criterion = nn.CrossEntropyLoss()

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [ ]:
model, optimizer, _ = training_loop(model, criterion, optimizer_arr[0], train_loader, test_loader, N_EPOCHS, DEVICE,LEARNING_RATE_arr[0])

11:25:43 --- Epoch: 0	Train loss: 2.5654	Test loss: 2.6472	Train accuracy: 9.31	Test accuracy: 9.17
11:26:06 --- Epoch: 1	Train loss: 3.1026	Test loss: 2.6099	Train accuracy: 9.87	Test accuracy: 9.80
11:26:29 --- Epoch: 2	Train loss: 3.1021	Test loss: 3.1693	Train accuracy: 9.86	Test accuracy: 9.58
11:26:52 --- Epoch: 3	Train loss: 2.9800	Test loss: 3.0832	Train accuracy: 9.03	Test accuracy: 8.92
11:27:15 --- Epoch: 4	Train loss: 3.0549	Test loss: 2.8349	Train accuracy: 9.91	Test accuracy: 10.09
11:27:40 --- Epoch: 5	Train loss: 3.0473	Test loss: 3.3575	Train accuracy: 9.74	Test accuracy: 9.82
11:28:02 --- Epoch: 6	Train loss: 3.0872	Test loss: 3.4742	Train accuracy: 9.93	Test accuracy: 10.32
11:28:26 --- Epoch: 7	Train loss: 3.0571	Test loss: 3.1494	Train accuracy: 11.24	Test accuracy: 11.35
11:28:50 --- Epoch: 8	Train loss: 3.0403	Test loss: 3.0363	Train accuracy: 9.75	Test accuracy: 9.74
11:29:13 --- Epoch: 9	Train loss: 3.0514	Test loss: 2.9437	Train accuracy: 9.87	Test accuracy: 9

In [ ]:
model, optimizer, _ = training_loop(model, criterion, optimizer_arr[1], train_loader, test_loader, N_EPOCHS, DEVICE,LEARNING_RATE_arr[1])

11:17:27 --- Epoch: 0	Train loss: 0.2770	Test loss: 0.2105	Train accuracy: 93.32	Test accuracy: 93.74
11:17:51 --- Epoch: 1	Train loss: 0.2065	Test loss: 0.1776	Train accuracy: 94.38	Test accuracy: 94.39
11:18:14 --- Epoch: 2	Train loss: 0.1909	Test loss: 0.2186	Train accuracy: 93.93	Test accuracy: 93.71
11:18:37 --- Epoch: 3	Train loss: 0.1986	Test loss: 0.1739	Train accuracy: 94.79	Test accuracy: 94.89
11:19:00 --- Epoch: 4	Train loss: 0.1820	Test loss: 0.1598	Train accuracy: 95.13	Test accuracy: 95.48
11:19:22 --- Epoch: 5	Train loss: 0.1792	Test loss: 0.1721	Train accuracy: 94.75	Test accuracy: 95.36
11:19:45 --- Epoch: 6	Train loss: 0.1848	Test loss: 0.1831	Train accuracy: 94.75	Test accuracy: 94.67
11:20:08 --- Epoch: 7	Train loss: 0.1895	Test loss: 0.1683	Train accuracy: 95.32	Test accuracy: 95.34
11:20:31 --- Epoch: 8	Train loss: 0.1663	Test loss: 0.1897	Train accuracy: 94.78	Test accuracy: 94.91
11:20:54 --- Epoch: 9	Train loss: 0.1818	Test loss: 0.1790	Train accuracy: 94.49	T

In [ ]:
model, optimizer, _ = training_loop(model, criterion, optimizer_arr[2], train_loader, test_loader, N_EPOCHS, DEVICE,LEARNING_RATE_arr[2])

15:28:01 --- Epoch: 0	Train loss: 0.2290	Valid loss: 0.1020	Train accuracy: 96.84	Valid accuracy: 96.81
15:28:25 --- Epoch: 1	Train loss: 0.0762	Valid loss: 0.0619	Train accuracy: 98.39	Valid accuracy: 98.19
15:28:48 --- Epoch: 2	Train loss: 0.0550	Valid loss: 0.0542	Train accuracy: 98.59	Valid accuracy: 98.45
15:29:12 --- Epoch: 3	Train loss: 0.0438	Valid loss: 0.0486	Train accuracy: 99.07	Valid accuracy: 98.41
15:29:35 --- Epoch: 4	Train loss: 0.0343	Valid loss: 0.0416	Train accuracy: 99.24	Valid accuracy: 98.76
15:29:58 --- Epoch: 5	Train loss: 0.0300	Valid loss: 0.0442	Train accuracy: 99.43	Valid accuracy: 98.58
15:30:21 --- Epoch: 6	Train loss: 0.0249	Valid loss: 0.0427	Train accuracy: 99.49	Valid accuracy: 98.70
15:30:44 --- Epoch: 7	Train loss: 0.0213	Valid loss: 0.0436	Train accuracy: 99.53	Valid accuracy: 98.64
15:31:07 --- Epoch: 8	Train loss: 0.0181	Valid loss: 0.0462	Train accuracy: 99.51	Valid accuracy: 98.72
15:31:31 --- Epoch: 9	Train loss: 0.0176	Valid loss: 0.0491	Trai

In [ ]:
!tensorboard dev upload --logdir 'runs' --name "MNIST" --description "LeNet MNIST for K=3 and N=10"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/InDPc00zQ4KtNqwcm26WFg/

[2022-06-19T11:31:44] Started scanning logdir.
[2022-06-19T11:31:44] Total uploaded: 120 scalars, 0 tensors, 0 binary objects


Done. View your TensorBoard at https://tensorboard.dev/experiment/InDPc00zQ4KtNqwcm26WFg/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/uploader/uploader_subcommand.py", line 482, in execute
    uploader.start_uploading()
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/uploader/uploader.py", line 212, in start_uploading
    self._logdir_poll_rate_limiter.tick()
  File "/usr/local/lib/python3.7/dist-packages/tensorboard/uploader/util.py", line 42, in tick
    self._time.sleep(wait_secs)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceb

### **LeNET CIFAR10 Dataset**

In [2]:
train_augmentations = transforms.Compose([transforms.RandomGrayscale(0.2),
                                          transforms.RandomHorizontalFlip(0.5),
                                          transforms.RandomVerticalFlip(0.2),
                                          transforms.RandomRotation(30),
                                          transforms.RandomAdjustSharpness(0.4),
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
                                         ])

test_augmentations = transforms.Compose([transforms.ToTensor(), 
                                         transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
                                       )

In [3]:
path = '/tmp'

train_ds = datasets.CIFAR10(root=path,
                            train=True,
                            transform=train_augmentations,
                            download=True
                           )

test_ds = datasets.CIFAR10(root=path,
                           train=False,
                           transform=test_augmentations
                           )

labels = 'airplane automobile bird cat deer dog frog horse ship truck'.split()

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting /tmp/cifar-10-python.tar.gz to /tmp


In [4]:
len(train_ds), len(test_ds)

(50000, 10000)

In [5]:
train_dataloader = torch.utils.data.DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=2)
test_dataloader = torch.utils.data.DataLoader(test_ds, batch_size=256, shuffle=True, num_workers=2)

In [6]:
class LeNet(nn.Module):
    
    def __init__(self, in_features=3, num_classes=10):
        super(LeNet, self).__init__()
        
        self.conv_block = nn.Sequential( nn.Conv2d(in_channels=in_features,
                                                   out_channels=6,
                                                   kernel_size=5,
                                                   stride=1),
                                         nn.Tanh(),
                                         nn.MaxPool2d(2,2),
                                         
                                         nn.Conv2d(in_channels=6,
                                                   out_channels=16,
                                                   kernel_size=5,
                                                   stride=1),
                                         nn.Tanh(),
                                         nn.MaxPool2d(2,2)
                                        )
        
        self.linear_block = nn.Sequential( nn.Linear(16*5*5, 120),
                                           nn.Tanh(),
                                           nn.Linear(120,84),
                                           nn.Tanh(),
                                           nn.Linear(84,10)
                                         )
        
    def forward(self, x):
        x = self.conv_block(x)
        x = torch.flatten(x,1)
        x = self.linear_block(x)
        return x

In [7]:
# CUDA
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
model = LeNet().to(device)
print(model)

LeNet(
  (conv_block): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Tanh()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Tanh()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_block): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Tanh()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [13]:
class Trainer:
    def __init__(self, model, dataloaders, device,lr): 
        self.config = {
            'lr':lr,
            'epochs': 10
        }     
        self.model = model
        self.train_loader, self.test_loader = dataloaders
        self.loss_fn = nn.CrossEntropyLoss()
        self.optim = torch.optim.Adam(self.model.parameters(), lr = self.config['lr'])
        self.device = device      
    def accuracy(self, output, y):
        pred_labels = torch.argmax(output, dim=1)
        return (pred_labels == y).sum().item() / len(y)     
    def train_one_epoch(self):
        running_loss = 0
        running_acc = 0 
        for x,y in self.train_loader:
            self.optim.zero_grad()
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)
            output = self.model(x)
            loss = self.loss_fn(output, y)
            loss.backward()
            self.optim.step()
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)
            del x,y,output
        train_loss = running_loss/len(self.train_loader)
        train_acc = running_acc/len(self.train_loader)
        return train_loss, train_acc
    
    @torch.no_grad()
    def valid_one_epoch(self):      
        running_loss = 0
        running_acc = 0       
        for x,y in self.test_loader:           
            x = x.to(self.device, dtype=torch.float)
            y = y.to(self.device, dtype=torch.long)          
            output = self.model(x)          
            loss = self.loss_fn(output, y)          
            running_loss += loss.item()
            running_acc += self.accuracy(output,y)           
            del x,y,output           
        test_loss = running_loss/len(self.test_loader)
        test_acc = running_acc/len(self.test_loader)     
        return test_loss, test_acc
    
    
    def fit(self,lr):
        
        train_losses,train_accs = [], []
        test_losses, test_accs = [], []     
        for epoch in range(self.config['epochs']):        
            print(f"Model is using {'cuda' if next(self.model.parameters()).is_cuda else 'cpu'}")            
            self.model.train()            
            train_loss, train_acc = self.train_one_epoch()
            tb.add_scalar("Loss/Training_Loss", train_loss, epoch)
            tb.add_scalar("Loss/Training_Accuracy", train_acc, epoch)
            train_losses.append(train_loss)
            train_accs.append(train_acc)           
            self.model.eval()           
            test_loss, test_acc = self.valid_one_epoch()
            tb.add_scalar("Loss/Testing_Loss", test_loss, epoch)
            tb.add_scalar("Loss/Testing_Accuracy", test_acc, epoch)
            test_losses.append(test_loss)
            test_accs.append(test_acc)           
            print(f"------EPOCH {epoch+1}/{self.config['epochs']}------")
            print(f"Training: LOSS: {train_loss} | ACCURACY: {train_acc} | for Learning Rate: {lr} ")
            print(f"Testing: LOSS: {test_loss} | ACCURACY: {test_acc} | for Learning Rate: {lr} \n\n")            
            # CLEANUP
            gc.collect()
            torch.cuda.empty_cache()
            tb.flush()       
        return (train_losses, train_accs), (test_losses, test_accs)

In [18]:
lr_arr=[0.1,0.001,0.0001]
trainer = Trainer(model, (train_dataloader, test_dataloader), device,lr_arr[0])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(lr_arr[0])

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 2.4089305784021104 | ACCURACY: 0.1766621492346939 | for Learning Rate: 0.1 
Testing: LOSS: 2.2927896857261656 | ACCURACY: 0.18447265625 | for Learning Rate: 0.1 


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 2.330635433294335 | ACCURACY: 0.18291214923469387 | for Learning Rate: 0.1 
Testing: LOSS: 2.3063986241817473 | ACCURACY: 0.2056640625 | for Learning Rate: 0.1 


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 2.332407368689167 | ACCURACY: 0.18478954081632654 | for Learning Rate: 0.1 
Testing: LOSS: 2.389987474679947 | ACCURACY: 0.186328125 | for Learning Rate: 0.1 


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 2.3427180976283792 | ACCURACY: 0.184375 | for Learning Rate: 0.1 
Testing: LOSS: 2.2863138735294344 | ACCURACY: 0.19091796875 | for Learning Rate: 0.1 


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 2.3808154646231205 | ACCURACY: 0.18144929846938776 | for Learni

In [16]:
trainer = Trainer(model, (train_dataloader, test_dataloader), device,lr_arr[1])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(lr_arr[1])

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.6064394919239744 | ACCURACY: 0.41929607780612244 | for Learning Rate: 0.001 
Testing: LOSS: 1.4127353042364121 | ACCURACY: 0.49033203125 | for Learning Rate: 0.001 


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.557299638281063 | ACCURACY: 0.440593112244898 | for Learning Rate: 0.001 
Testing: LOSS: 1.3649799168109893 | ACCURACY: 0.50703125 | for Learning Rate: 0.001 


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.510778386373909 | ACCURACY: 0.455110012755102 | for Learning Rate: 0.001 
Testing: LOSS: 1.3390962719917296 | ACCURACY: 0.5234375 | for Learning Rate: 0.001 


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.4826122418958314 | ACCURACY: 0.46646205357142856 | for Learning Rate: 0.001 
Testing: LOSS: 1.3303478956222534 | ACCURACY: 0.5240234375 | for Learning Rate: 0.001 


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.4538150685174125 | ACCURACY: 0.4781768176

In [14]:
lr_arr=[0.1,0.001,0.0001]
trainer = Trainer(model, (train_dataloader, test_dataloader), device,lr_arr[2])
(train_losses, train_accs), (test_losses, test_accs) = trainer.fit(lr_arr[2])

Model is using cuda
------EPOCH 1/10------
Training: LOSS: 1.9163538758852043 | ACCURACY: 0.3180763711734694 | for Learning Rate: 0.0001 
Testing: LOSS: 1.7766589254140854 | ACCURACY: 0.37578125 | for Learning Rate: 0.0001 


Model is using cuda
------EPOCH 2/10------
Training: LOSS: 1.8468560837969488 | ACCURACY: 0.3402582908163265 | for Learning Rate: 0.0001 
Testing: LOSS: 1.706892666220665 | ACCURACY: 0.403125 | for Learning Rate: 0.0001 


Model is using cuda
------EPOCH 3/10------
Training: LOSS: 1.7898331333179862 | ACCURACY: 0.35825892857142855 | for Learning Rate: 0.0001 
Testing: LOSS: 1.6427015662193298 | ACCURACY: 0.41826171875 | for Learning Rate: 0.0001 


Model is using cuda
------EPOCH 4/10------
Training: LOSS: 1.7464979941747627 | ACCURACY: 0.372444993622449 | for Learning Rate: 0.0001 
Testing: LOSS: 1.5878644108772277 | ACCURACY: 0.434375 | for Learning Rate: 0.0001 


Model is using cuda
------EPOCH 5/10------
Training: LOSS: 1.7103755103082072 | ACCURACY: 0.383438

In [11]:
from torch.utils.tensorboard import SummaryWriter
import torchvision
# load data from loader
images, labels = next(iter(train_dataloader))
images, labels = images.cuda(), labels.cuda()
grid = torchvision.utils.make_grid(images)

tb = SummaryWriter()
tb.add_image('images', grid)
tb.add_graph(model, images)
tb.close()

In [19]:
!tensorboard dev upload --logdir 'runs' --name "CIFAR 10 Data " --description "LeNet CIFAR10 Graph"

Upload started and will continue reading any new data as it's added to the logdir.

To stop uploading, press Ctrl-C.

New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/wvqriaZPQtetA4jvpwOXyA/

[2022-06-19T12:32:37] Started scanning logdir.
[2022-06-19T12:32:38] Total uploaded: 124 scalars, 0 tensors, 1 binary objects (8.6 kB)


Interrupted. View your TensorBoard at https://tensorboard.dev/experiment/wvqriaZPQtetA4jvpwOXyA/
Exception ignored in: <function Channel.__del__ at 0x7f9790d117a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/grpc/_channel.py", line 1570, in __del__
    def __del__(self):
KeyboardInterrupt
